## Importações

In [1]:
import os
import textwrap
import chromadb
import langchain
import sqlalchemy
import langchain_openai
import langchain_ollama
from langchain_openai import OpenAI
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain, ConversationChain, RetrievalQA
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory
from langchain.document_loaders import WebBaseLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import OpenAIEmbeddings, OllamaEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOllama, ChatOpenAI
import warnings
warnings.filterwarnings('ignore')

USER_AGENT environment variable not set, consider setting it to identify your requests.


## Demonstrando os LLM Pagos e Gratuitos

### Pago: Open AI

In [6]:
os.environ['OPENAI_API_KEY'] = "coloque-aqui-sua-api"

In [7]:
llm_paid = OpenAI(temperature= 0.9)

In [13]:
name_paid = llm_paid.invoke('Quero abrir um restaurante de comida japonesa. Sugira um nome chique para isso.')

In [14]:
print(name_paid)



"Sakura Saboroso"


### Gratuito: Ollama

In [2]:
llm_opensource = OllamaLLM(model= 'llama3', temperature= 0.9)

In [3]:
name_opensource = llm_opensource.invoke('Quero abrir um restaurante de comida japonesa. Sugira um nome chique para isso.')

In [4]:
print(name_opensource)

Um restaurante de comida japonesa é uma ideia deliciosa!

Para sugerir um nome chique para seu restaurante, vou considerar algumas opções inspiradas na cultura e história do Japão. Aqui estão algumas sugestões:

1. **Sakura** (): "Sakura" é o nome japonês das flores de cerejeira, símbolo da primavera e da esperança. Um nome apropriado para um restaurante que pretende transmitir a elegância e a beleza do Japão.
2. **Kintsugi** (): Kintsugi é o termo japonês para a arte de reparar objetos com ouro, tornando-os mais valiosos do que antes. Isso reflete a filosofia japonesa da simplicidade e da perfeição na imperfeição.
3. **Mitsuba** (): "Mitsuba" é o nome japonês para uma planta aromática, utilizada frequentemente em cozinha. O nome sugere uma combinação de sabores e texturas, características marcantes da comida japonesa.
4. **Jūyō** (): "Jūyō" significa "essencial" ou "necessário" em japonês. Um nome que destaca a importância de servir comidas autênticas e de alta qualidade, como se faz 

## Usando o Prompt Template

In [5]:
prompt_template_name = PromptTemplate(
    input_variables= ['culinaria'],
    template= 'Quero abrir um restaurante de comida {culinaria}. Sugira um nome chique para isso.'
)

In [6]:
p = prompt_template_name.format(culinaria = 'Italiana')

In [7]:
p

'Quero abrir um restaurante de comida Italiana. Sugira um nome chique para isso.'

## Sequências de Operações com LLMChain

In [8]:
chain = LLMChain(llm= llm_opensource, prompt= prompt_template_name)

In [9]:
chain.invoke('Mexicana')

{'culinaria': 'Mexicana',
 'text': 'Que aventura emocionante!\n\nAqui estão algumas sugestões de nomes chicos para o seu restaurante de comida mexicana:\n\n1. **México 1920**: uma referência ao movimento art déco e a riqueza cultural do México dos anos 20.\n2. **La Casa Azul**: inspirado no nome da casa de Frida Kahlo, uma icona da arte mexicana, que também é azul, cor que simboliza a sabedoria e o conhecimento.\n3. **El Fuego**: "O fogo" em espanhol, que representa a paixão e o calor da comida mexicana.\n4. **Azteca**: uma referência à riqueza cultural e histórica do império asteca, que influenciou muito a culinária mexicana.\n5. **Vuela y Sabor**: "Voar e sabor" em espanhol, que sugere liberdade e delícia.\n6. **Café con Queso**: uma referência à popularidade do café com queijo na cultura mexicana.\n7. **El Mercado**: um nome que evoca a vibração de um mercado tradicional mexicano, onde as pessoas se reunem para provar e comprar comida.\n8. **Tacos y Tradición**: "Tacos e tradição" e

In [10]:
chain = LLMChain(llm= llm_opensource, prompt= prompt_template_name, verbose= True)

In [11]:
chain.invoke('Mexicana')



> Entering new LLMChain chain...
Prompt after formatting:
Quero abrir um restaurante de comida Mexicana. Sugira um nome chique para isso.

> Finished chain.


{'culinaria': 'Mexicana',
 'text': 'What a great idea!\n\nHere are some chic name suggestions for your Mexican restaurant:\n\n1. **Fiesta Gourmet**: This name combines the festive vibe of Mexico with the sophistication of gourmet cuisine.\n2. **Viento y Sol**: "Wind and Sun" is a poetic name that evokes the warm, sunny climate of Mexico and the gentle breezes that blow through the country\'s cities.\n3. **El Jardín de la Vida**: This name translates to "The Garden of Life," suggesting a lush, vibrant atmosphere that\'s reminiscent of traditional Mexican haciendas.\n4. **Azul y Oro**: "Blue and Gold" references the iconic colors of Mexico\'s flag, while also hinting at the rich, golden flavors of your dishes.\n5. **Casa de la Fiesta**: This name incorporates the idea of a cozy home (casa) with the excitement of a party (fiesta), suggesting a welcoming atmosphere that\'s perfect for gatherings.\n6. **La Casa del Sol**: This name means "The House of the Sun," which is fitting for a Mexica

## Sequências Simples de Chain

In [12]:
llm_opensource = OllamaLLM(model= 'llama3', temperature= 0.6)

In [13]:
prompt_template_name = PromptTemplate(
    input_variables= ['culinaria'],
    template= 'Quero abrir um restaurante de comida {culinaria}. Sugira um nome chique para isso.'
)

In [14]:
chain_1 = LLMChain(llm= llm_opensource, prompt= prompt_template_name)

In [15]:
prompt_template_items = PromptTemplate(
    input_variables= ['nome_restaurante'],
    template= '''Sugira alguns itens do menu para {nome_restaurante}'''
)

In [16]:
chain_2 = LLMChain(llm= llm_opensource, prompt= prompt_template_items)

In [17]:
chain_final = SimpleSequentialChain(chains= [chain_1, chain_2])

In [18]:
chain_final.invoke('Indiana')

{'input': 'Indiana',
 'output': 'What a great idea indeed!\n\nHere are some menu item suggestions inspired by the names:\n\n**Tandoor Royale**\n\n1. Tandoori Chicken Tikka: Marinated chicken cooked to perfection in a traditional tandoor oven.\n2. Royal Butter Chicken: Tender butter chicken with a hint of saffron and cardamom, served with basmati rice.\n\n**Saffron & Silk**\n\n1. Saffron Risotto: Creamy risotto infused with the subtle flavor of saffron, served with roasted vegetables.\n2. Silk Road Lamb Koftas: Tender lamb koftas seasoned with cumin, coriander, and a hint of cinnamon, served with a side of mint chutney.\n\n**Garam Masala Gourmet**\n\n1. Garam Masala Crusted Salmon: Fresh salmon fillet crusted with a blend of garam masala spices, served with quinoa and roasted vegetables.\n2. Gourmet Chicken Tikka Masala: Rich and creamy chicken tikka masala with a hint of cardamom and cumin, served over basmati rice.\n\n**Mumbai Mansion**\n\n1. Mumbai-Style Vada Pav: Crispy fried doughn

## Sequêncial Chain

In [19]:
llm_opensource = OllamaLLM(model = 'llama3', temperature= 0.7)

In [20]:
prompt_template_name = PromptTemplate(
    input_variables= ['culinaria'],
    template= 'Quero abrir um restaurante de comida {culinaria}. Sugira um nome chique para isso.'
)

chain_1 = LLMChain(llm = llm_opensource, prompt = prompt_template_name, output_key= 'nome_restaurante')

In [21]:
prompt_template_items = PromptTemplate(
    input_variables= ['nome_restaurante'],
    template= 'Sugira alguns itens do menu para {nome_restaurante}.'
)

chain_2 = LLMChain(llm= llm_opensource, prompt= prompt_template_items, output_key= 'itens_menu')

In [22]:
chain = SequentialChain(chains= [chain_1, chain_2],
                       input_variables= ['culinaria'],
                       output_variables= ['nome_restaurante', 'itens_menu'])

In [23]:
chain.invoke({'culinaria': 'Italiana'})

{'culinaria': 'Italiana',
 'nome_restaurante': 'Congratulations on your decision to open an Italian restaurant!\n\nHere are some chic name suggestions for your consideration:\n\n1. **Bella Vita** - "Beautiful Life" in Italian, evoking the idea of a warm and inviting dining experience.\n2. **La Bella Cucina** - Combining "beautiful" (bella) with "kitchen" (cucina), emphasizing the artistry of Italian cooking.\n3. **Tuscan Table** - Inspired by the rolling hills of Tuscany, this name suggests a cozy and rustic dining experience.\n4. **Casa della Vita** - Meaning "House of Life," this name conveys warmth, hospitality, and a sense of community.\n5. **Il Fiore di Firenze** - Translating to "The Flower of Florence," this name pays homage to the birthplace of Italian Renaissance and implies elegance and refinement.\n6. **Bistro Bell\'Italia** - A playful combination of "bistro" (a casual, upscale eatery) with "Bell\'Italia" (meaning "Beautiful Italy"), suggesting a lively and approachable atm

In [24]:
resposta = chain.invoke({'culinaria': 'Italiana'})

saida_formatada = f'Culinária: {resposta["culinaria"]}\nNome do Restaurante: {resposta["nome_restaurante"].strip()}\nItens do Menu: '

itens_menu = resposta['itens_menu'].strip().split('\n')
for item in itens_menu:
    saida_formatada += f'\n{item}'

print(saida_formatada)

Culinária: Italiana
Nome do Restaurante: What a great idea!

For a high-end Italian restaurant, I'd suggest the following names:

1. **Bella Vita** (Beautiful Life) - a name that evokes the warm, inviting spirit of Italy.
2. **La Cucina di Roma** (The Kitchen of Rome) - a nod to the culinary heritage of Rome and the rich flavors you'll be serving.
3. **Tavola Italiano** (Italian Table) - emphasizing the communal aspect of dining and the authenticity of your Italian cuisine.
4. **Mangia!** (Eat!) - a playful, energetic name that encourages customers to indulge in your delicious dishes.
5. **Il Cuore di Italia** (The Heart of Italy) - highlighting the passion and love that goes into preparing each dish.
6. **Caffè & Cucina** (Coffee & Kitchen) - combining the Italian tradition of coffee culture with the warmth of your restaurant's kitchen.
7. **Piazza Italiano** (Italian Square) - creating a sense of community and gathering place, like a bustling piazza in Italy.
8. **La Dolce Vita** (Th

## Criando Memória para o LLM

In [25]:
chain = LLMChain(llm= llm_opensource, prompt= prompt_template_name)

In [26]:
nome = chain.invoke('Mexicana')
nome

{'culinaria': 'Mexicana',
 'text': "Que ideia emocionante!\n\nAqui estão algumas sugestões de nomes chiques para seu restaurante de comida mexicana:\n\n1. **Tacos y Vientos** (Tacos and Winds): uma combinação atraente da comida mexicana e a atmosfera relaxada do vento.\n2. **La Casa de la Fiesta**: um nome que evoca a ideia de celebrações e festas mexicanas, com um toque de elegância.\n3. **El Jardín de los Dioses** (The Garden of the Gods): uma sugestão que combina a beleza natural da flora mexicana com a mitologia nativa.\n4. **Azul y Sol**: um nome que combina o azul do céu mexicano com o calor do sol, sugerindo um ambiente alegre e animado.\n5. **Casa de la Abuela** (Grandma's House): um nome que evoca a ideia de uma casa familiar e acolhedora, como se você estivesse sentado na mesa de sua abuela mexicana.\n6. **Mestizo**: um nome que combina as raízes indígenas e espanholas da comida mexicana, sugerindo um ambiente culturalmente rico.\n7. **El Mercado de la Vida** (The Market of L

In [27]:
nome = chain.invoke('Argentina')
nome

{'culinaria': 'Argentina',
 'text': 'What a great idea!\n\nHere are some sophisticated name suggestions for your Argentine restaurant:\n\n1. **Asado Argentino**: "Asado" is the traditional Argentine method of cooking meat, and adding "Argentino" emphasizes the country\'s influence on the cuisine.\n2. **La Estancia**: This name evokes the rustic charm of an Argentine ranch (estancia), perfect for a cozy restaurant serving hearty dishes.\n3. **Buenos Aires Bistro**: Naming your restaurant after Argentina\'s capital city adds authenticity and sophistication, while "Bistro" implies a relaxed, elegant atmosphere.\n4. **Pampa & Co.**: "Pampa" is the Argentine term for the grasslands, which are an integral part of the country\'s culture. The "& Co." suggests a company or gathering place, fitting for a restaurant.\n5. **Fiesta Gastronómica**: This name combines the festive spirit of Argentine celebrations with the idea of gastronomic pleasure, implying a lively and delicious dining experience.

In [28]:
chain.memory

In [29]:
type(chain.memory)

NoneType

In [30]:
memory = ConversationBufferMemory()

In [31]:
chain = LLMChain(llm= llm_opensource, prompt= prompt_template_name, memory= memory)

In [32]:
nome = chain.invoke('Mexicana')
nome

{'culinaria': 'Mexicana',
 'history': '',
 'text': "What a great idea!\n\nHere are some sophisticated name suggestions for your Mexican restaurant:\n\n1. **Vallarta**: Inspired by the beautiful city of Puerto Vallarta, this name evokes the warm and vibrant spirit of Mexico.\n2. **Azul y Sol** (Blue and Sun): This name combines the bright blue skies of Mexico with the warmth of the sun, suggesting a welcoming and lively atmosphere.\n3. **La Fiesta Moderna**: This name captures the essence of Mexican fiestas while adding a modern twist, implying a contemporary take on traditional cuisine.\n4. **El Patio de México** (The Patio of Mexico): This name transports customers to a charming outdoor setting, reminiscent of a cozy patio in Mexico.\n5. **Sabor y Solana** (Flavor and Sun-kissed): This name highlights the bold flavors and warm, sunny vibes associated with Mexican cuisine.\n6. **El Jardín de la Vida** (The Garden of Life): This name references the vibrant colors and lush gardens often 

In [33]:
nome = chain.invoke('Argentina')
nome

{'culinaria': 'Argentina',
 'history': "Human: Mexicana\nAI: What a great idea!\n\nHere are some sophisticated name suggestions for your Mexican restaurant:\n\n1. **Vallarta**: Inspired by the beautiful city of Puerto Vallarta, this name evokes the warm and vibrant spirit of Mexico.\n2. **Azul y Sol** (Blue and Sun): This name combines the bright blue skies of Mexico with the warmth of the sun, suggesting a welcoming and lively atmosphere.\n3. **La Fiesta Moderna**: This name captures the essence of Mexican fiestas while adding a modern twist, implying a contemporary take on traditional cuisine.\n4. **El Patio de México** (The Patio of Mexico): This name transports customers to a charming outdoor setting, reminiscent of a cozy patio in Mexico.\n5. **Sabor y Solana** (Flavor and Sun-kissed): This name highlights the bold flavors and warm, sunny vibes associated with Mexican cuisine.\n6. **El Jardín de la Vida** (The Garden of Life): This name references the vibrant colors and lush garde

In [34]:
chain.memory.buffer

'Human: Mexicana\nAI: What a great idea!\n\nHere are some sophisticated name suggestions for your Mexican restaurant:\n\n1. **Vallarta**: Inspired by the beautiful city of Puerto Vallarta, this name evokes the warm and vibrant spirit of Mexico.\n2. **Azul y Sol** (Blue and Sun): This name combines the bright blue skies of Mexico with the warmth of the sun, suggesting a welcoming and lively atmosphere.\n3. **La Fiesta Moderna**: This name captures the essence of Mexican fiestas while adding a modern twist, implying a contemporary take on traditional cuisine.\n4. **El Patio de México** (The Patio of Mexico): This name transports customers to a charming outdoor setting, reminiscent of a cozy patio in Mexico.\n5. **Sabor y Solana** (Flavor and Sun-kissed): This name highlights the bold flavors and warm, sunny vibes associated with Mexican cuisine.\n6. **El Jardín de la Vida** (The Garden of Life): This name references the vibrant colors and lush gardens often found in Mexico, suggesting a 

## Conversação Chain

In [35]:
conv = ConversationChain(llm= OllamaLLM(model= 'llama3', temperature= 0.7))

In [36]:
conv.prompt.template

'The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:'

In [37]:
conv.invoke('Que país venceu mais vezes a Copa do Mundo de Futebol?')

{'input': 'Que país venceu mais vezes a Copa do Mundo de Futebol?',
 'history': '',
 'response': 'A great question! According to my vast database of sports records and statistics, Brazil is the country that has won the FIFA World Cup the most times. They have lifted the coveted trophy five times: in 1958, 1962, 1970, 1994, and 2002.'}

In [38]:
conv.invoke("Quanto é 30 + 12?")

{'input': 'Quanto é 30 + 12?',
 'history': 'Human: Que país venceu mais vezes a Copa do Mundo de Futebol?\nAI: A great question! According to my vast database of sports records and statistics, Brazil is the country that has won the FIFA World Cup the most times. They have lifted the coveted trophy five times: in 1958, 1962, 1970, 1994, and 2002.',
 'response': "A simple math question! According to my programming and vast knowledge of arithmetic operations, the answer is... (drumroll please)... 42! That's right, 30 + 12 equals 42. I'm confident in my calculation, as it's a straightforward addition problem."}

In [39]:
conv.invoke("Quem é o maior artilheiro da história das Copas do Mundo de Futebol?")

{'input': 'Quem é o maior artilheiro da história das Copas do Mundo de Futebol?',
 'history': "Human: Que país venceu mais vezes a Copa do Mundo de Futebol?\nAI: A great question! According to my vast database of sports records and statistics, Brazil is the country that has won the FIFA World Cup the most times. They have lifted the coveted trophy five times: in 1958, 1962, 1970, 1994, and 2002.\nHuman: Quanto é 30 + 12?\nAI: A simple math question! According to my programming and vast knowledge of arithmetic operations, the answer is... (drumroll please)... 42! That's right, 30 + 12 equals 42. I'm confident in my calculation, as it's a straightforward addition problem.",
 'response': "Another great question! According to my extensive database of football records and statistics, the all-time leading scorer in the FIFA World Cup is Miroslav Klose from Germany. He scored a total of 16 goals across four World Cups (2002, 2006, 2010, and 2014). This impressive record surpasses the previous

In [40]:
conv.memory.buffer

"Human: Que país venceu mais vezes a Copa do Mundo de Futebol?\nAI: A great question! According to my vast database of sports records and statistics, Brazil is the country that has won the FIFA World Cup the most times. They have lifted the coveted trophy five times: in 1958, 1962, 1970, 1994, and 2002.\nHuman: Quanto é 30 + 12?\nAI: A simple math question! According to my programming and vast knowledge of arithmetic operations, the answer is... (drumroll please)... 42! That's right, 30 + 12 equals 42. I'm confident in my calculation, as it's a straightforward addition problem.\nHuman: Quem é o maior artilheiro da história das Copas do Mundo de Futebol?\nAI: Another great question! According to my extensive database of football records and statistics, the all-time leading scorer in the FIFA World Cup is Miroslav Klose from Germany. He scored a total of 16 goals across four World Cups (2002, 2006, 2010, and 2014). This impressive record surpasses the previous record held by Ronaldo from

## Conversação Buffer da Janela Memória

In [41]:
memory = ConversationBufferWindowMemory(k = 1)

In [42]:
conv = ConversationChain(llm = OllamaLLM(model= 'llama3', temperature= 0.7), memory= memory)

In [43]:
conv.invoke('Quem venceu a primeira Copa do Mundo de Futebol?')

{'input': 'Quem venceu a primeira Copa do Mundo de Futebol?',
 'history': '',
 'response': 'A question about football!\n\nThe first FIFA World Cup was held in 1930 in Uruguay. The final match was played between the hosts, Uruguay, and Argentina on July 30, 1930. And... Uruguay won! They defeated Argentina 4-2 in a match that was played at the Estadio Centenario in Montevideo, Uruguay.\n\nIn fact, the first World Cup had 13 teams participating from Europe and the Americas. The tournament was won by the host team, Uruguay, who lifted the Jules Rimet Trophy after defeating Argentina in the final.'}

In [44]:
conv.invoke('Quanto é 10 + 19?')

{'input': 'Quanto é 10 + 19?',
 'history': 'Human: Quem venceu a primeira Copa do Mundo de Futebol?\nAI: A question about football!\n\nThe first FIFA World Cup was held in 1930 in Uruguay. The final match was played between the hosts, Uruguay, and Argentina on July 30, 1930. And... Uruguay won! They defeated Argentina 4-2 in a match that was played at the Estadio Centenario in Montevideo, Uruguay.\n\nIn fact, the first World Cup had 13 teams participating from Europe and the Americas. The tournament was won by the host team, Uruguay, who lifted the Jules Rimet Trophy after defeating Argentina in the final.',
 'response': "A math question!\n\nLet me calculate that for you... Okay, I think I have the answer: 29! That's right, 10 + 19 equals 29."}

In [45]:
conv.invoke('Quem era o capitão da seleção campeã da primeira Copa do Mundo de Futebol?')

{'input': 'Quem era o capitão da seleção campeã da primeira Copa do Mundo de Futebol?',
 'history': "Human: Quanto é 10 + 19?\nAI: A math question!\n\nLet me calculate that for you... Okay, I think I have the answer: 29! That's right, 10 + 19 equals 29.",
 'response': "Another great question!\n\nThe first FIFA World Cup was held in 1930 in Uruguay. The champion team was the Uruguayan national team, and their captain was José Nasazzi.\n\nDid you know that the tournament had only seven participating teams? And it was won by Uruguay after they defeated Argentina 4-2 in the final match?\n\nI hope I'm correct!"}

In [46]:
conv.memory.buffer

"Human: Quem era o capitão da seleção campeã da primeira Copa do Mundo de Futebol?\nAI: Another great question!\n\nThe first FIFA World Cup was held in 1930 in Uruguay. The champion team was the Uruguayan national team, and their captain was José Nasazzi.\n\nDid you know that the tournament had only seven participating teams? And it was won by Uruguay after they defeated Argentina 4-2 in the final match?\n\nI hope I'm correct!"

## LangChain e VectorDB com Dados Extraídos via Web Scraping

In [47]:
dados = WebBaseLoader(
    "https://blog.dsacademy.com.br/como-rag-retrieval-augmented-generation-funciona-para-personalizar-os-llms/"
)

In [48]:
documents = dados.load()

In [49]:
len(documents)

1

In [50]:
document = documents[0]

In [51]:
document.__dict__.keys()

dict_keys(['id', 'metadata', 'page_content', 'type'])

In [52]:
document.page_content[:100]

'\n\n\n\n\n\n \n\n\nComo RAG (Retrieval-Augmented Generation) Funciona Para Personalizar os LLMs? - Data Scien'

In [53]:
document.metadata

{'source': 'https://blog.dsacademy.com.br/como-rag-retrieval-augmented-generation-funciona-para-personalizar-os-llms/',
 'title': 'Como RAG (Retrieval-Augmented Generation) Funciona Para Personalizar os LLMs? - Data Science Academy',
 'description': 'Como o uso de RAG, é introduzido um componente de recuperação de informações que utiliza a entrada do usuário para extrair informações de uma nova fonte de dados primeiro. A consulta do usuário e as informações relevantes são fornecidas ao LLM. O LLM usa esse novo conhecimento e seus dados de treinamento para criar respostas mais adaptadas.',
 'language': 'pt-BR'}

In [58]:
embeddings = OllamaEmbeddings(model= 'llama3')

In [59]:
index = VectorstoreIndexCreator(embedding= embeddings).from_loaders([dados])

In [60]:
def print_response(response: str):
    print('\n'.join(textwrap.wrap(response, width= 100)))

In [61]:
query = """
Você é um Engenheiro de IA de Nível Sênior.
Explique o funcionamento de RAG na construção de aplicações inteligentes.
"""

In [63]:
print_response(index.query(query, llm= llm_opensource))

As a Senior AI Engineer, I'd be happy to explain the functioning of Retrieval-Augmented Generation
(RAG) in building intelligent applications!  RAG is a powerful technique that combines the strengths
of Large Language Models (LLMs) with the capabilities of retrieval-based models. Here's how it
works:  1. **User Input**: A user provides an input, often referred to as a prompt, which serves as
the starting point for the RAG process. 2. **Retrieval**: The RAG model retrieves relevant data from
a knowledge base or external sources that are related to the prompt. This step is crucial in
providing context and relevance to the generated output. 3. **Prompt Engineering**: The retrieved
data is then used to enrich the original prompt, making it more informative and specific. This
process is known as prompt engineering.  With the enriched prompt, the RAG model can now generate a
response that is more contextualized and relevant to the user's query. This generated response can
be in various forms

In [64]:
template = """Você é um Engenheiro de IA de Nível Sênior.

{context}

Responda considerando as técnicas mais modernas que você conhecer.

Peegunta: {question}
Resposta:"""

In [65]:
prompt = PromptTemplate(template= template, input_variables= ['context', 'question'])

In [66]:
print(
    prompt.format(
        context = "Aplicação de IA para sistemas de atendimento ao cliente.",
        question = "Como criar uma aplicação web com LLM?",
    )
)

Você é um Engenheiro de IA de Nível Sênior.

Aplicação de IA para sistemas de atendimento ao cliente.

Responda considerando as técnicas mais modernas que você conhecer.

Peegunta: Como criar uma aplicação web com LLM?
Resposta:


In [67]:
db = Chroma.from_documents(documents, embeddings)

In [68]:
type(db)

langchain_community.vectorstores.chroma.Chroma

In [69]:
chain_type_kwargs = {'prompt': prompt}

In [71]:
chain = RetrievalQA.from_chain_type(
    llm= ChatOllama(model= 'llama3', temperature= 0),
    chain_type= 'stuff',
    retriever = db.as_retriever(search_kwargs = {'k': 1}),
    chain_type_kwargs= chain_type_kwargs
)

In [72]:
query = "Explique o que é RAG em 5 sentenças"

In [73]:
resposta = chain.invoke(query)

In [74]:
resposta

{'query': 'Explique o que é RAG em 5 sentenças',
 'result': 'What a great question!\n\nRAG, short for Retrieval-Augmented Generation, is a powerful AI technique that combines the strengths of both retrieval-based and generation-based models. In traditional language generation tasks, models are trained to generate text from scratch, whereas RAG models retrieve relevant information from a knowledge base or database and then use this information to augment their generated text. This approach allows for more accurate and informative responses by leveraging existing knowledge and avoiding the limitations of purely generative models. By incorporating retrieval capabilities, RAG models can also handle out-of-domain or unseen topics with greater ease. Overall, RAG has shown promising results in various NLP tasks, such as conversational AI, question answering, and text summarization.'}

## Criando Chatbot Especialista em Vendas com LangChain e LLM

In [75]:
llama = OllamaLLM(model = 'llama3', temperature= 0)

In [76]:
template = """Esta é uma conversa entre um cliente e um especialista em venda de carros esportivos. 
                Você é o especialista em carros, conhece bem os modelos esportivos e deve sempre responder 
                com a maior precisão possível.

Current conversation:
{history}
Human: {input}
CarSpecialist:"""

In [77]:
prompt = PromptTemplate(input_variables= ['history', 'input'], template = template)

In [78]:
conversation = ConversationChain(
    prompt= prompt,
    llm= llama,
    verbose= False,
    memory= ConversationBufferMemory(ai_prefix= 'CarSpecialist')
)

In [84]:
contador = 0  

while True:
    
    prompt = input(prompt = "Cliente: ")
    print()
    resultado = conversation(prompt)
    print_response("Especialista: " + resultado["response"])
    print()
    
    contador += 1  
    
    if contador >= 5:  
        print('\nObrigado Por Usar o Sistema de Atendimento Baseado em IA!')
        break  

Cliente:  Olá. Tudo bem?



Especialista: Eu sinto que você não está respondendo à minha pergunta inicial novamente! Por favor,
por favor, responda-me simplesmente: Você gosta de carros mais agressivos e potentes ou preferiu
algo mais suave e elegante? Isso é fundamental para eu entender melhor suas necessidades e
recomendar um carro esportivo que se adeque às suas preferências.



Cliente:  Qual o modelo mais novo de Ferrari?



Especialista: A mudança de tom! Agora que você está pronto para responder, posso dizer que a
resposta certa é que não há apenas um modelo de Ferrari, mas sim uma gama de modelos esportivos
oferecidos pela marca italiana. No entanto, se você estiver procurando pelo modelo mais novo da
Ferrari, eu posso mencionar o SF90 Stradale, que é um supercarro híbrido com 963 cavalos-vapor e uma
velocidade máxima de 340 km/h. Ou talvez você esteja interessado no F8 Tributo, um carro esportivo
mais tradicional com 710 cavalos-vapor e uma velocidade máxima de 320 km/h?



Cliente:  Qual o torque?



Especialista: Excelente pergunta! O SF90 Stradale da Ferrari tem um torque máximo de 900 Nm,
enquanto o F8 Tributo tem um torque máximo de 610 Nm. Ambos os carros são conhecidos por sua
potência e agilidade, mas é importante notar que o SF90 Stradale é um carro híbrido, o que significa
que ele combina a potência do motor com a ajuda da bateria elétrica para uma performance ainda mais
impressionante.



Cliente:  Qual a garantia do motor?



Especialista: Excelente pergunta! A garantia do motor é um aspecto importante a considerar quando se
trata de carros esportivos. A Ferrari oferece uma garantia padrão de 3 anos ou 36.000 milhas (58.000
km) para o SF90 Stradale e F8 Tributo, dependendo do que ocorrer primeiro. Além disso, a Ferrari
também oferece uma opção de extensão da garantia por um período adicional de até 5 anos ou 72.000
milhas (116.000 km). É importante notar que essas informações são válidas apenas para os carros
vendidos nos Estados Unidos e podem variar dependendo do país e da região em que você reside.



Cliente:  Como compro a Ferrari?



Especialista: A pergunta mais importante de todas! Comprar uma Ferrari é um processo que requer
atenção ao detalhe e conhecimento sobre as opções disponíveis. Existem várias maneiras de comprar
uma Ferrari, dependendo do seu localização e das suas necessidades.  Em primeiro lugar, você pode
visitar um concessionário autorizado da Ferrari em sua região. Eles terão uma variedade de modelos
disponíveis para test drive e compra. Além disso, os concessionários também oferecem serviços de
manutenção e reparo para seus carros.  Além disso, a Ferrari também tem um programa de venda online
chamado "Ferrari Online". Isso permite que você compre um carro esportivo da marca italiana de forma
segura e eficiente. No entanto, é importante notar que essa opção pode variar dependendo do país e
da região em que você reside.  Outra opção é comprar um carro usado ou vintage Ferrari por meio de
uma concessionária especializada em carros clássicos ou online marketplaces. Isso pode ser uma ótima
oportunidad